# Instance - IRIS Operations Agent: Complete Workflow Demo

This notebook demonstrates the full workflow of the Instance system:
1. Error Generation (User Story 1)
2. Message Output (User Story 2)
3. Command Reception (User Story 3)
4. Orchestration & Execution (User Stories 4-6)

## Setup

In [3]:
# Configure environment
import sys
from pathlib import Path

# Find project root by looking for src/ directory
current_dir = Path.cwd()
project_root = current_dir

# If we're in notebooks/demos/, go up 2 levels
if current_dir.name == 'demos' and current_dir.parent.name == 'notebooks':
    project_root = current_dir.parent.parent
# If we're in notebooks/, go up 1 level
elif current_dir.name == 'notebooks':
    project_root = current_dir.parent
# Otherwise, check if src/ exists here
elif not (current_dir / 'src').exists():
    # Try going up until we find src/
    for parent in current_dir.parents:
        if (parent / 'src').exists():
            project_root = parent
            break

sys.path.insert(0, str(project_root))

# Configure logging
from src.utils.logger import configure_logging, get_logger
configure_logging(log_level="INFO", log_format="console")
logger = get_logger("demo")

print(f"✓ Project root: {project_root}")
print(f"✓ Python path: {sys.path[0]}")
print(f"✓ Logging configured")

✓ Project root: c:\tmp\Hack2
✓ Python path: c:\tmp\Hack2
✓ Logging configured


## Demo 1: Error Generation

Generate realistic IRIS error messages based on actual log patterns.

In [4]:
from src.agents.error_generator import ErrorGeneratorAgent
from src.models.error_message import ErrorGenerationRequest

# Initialize agent with sample log
log_path = project_root / "tests" / "fixtures" / "sample_messages.log"
agent = ErrorGeneratorAgent(log_samples_path=log_path)

# Generate a license error
request = ErrorGenerationRequest(
    error_category="license",
    severity=2
)

error = agent.generate_error(request)

print("Generated Error Message:")
print("=" * 80)
print(error.to_log_format())
print("\nDetails:")
print(f"  Timestamp: {error.timestamp}")
print(f"  Process ID: {error.process_id}")
print(f"  Severity: {error.severity} (0=info, 1=warning, 2=error, 3=fatal)")
print(f"  Category: {error.category}")
print(f"  Message: {error.message_text}")

2026-02-06T16:29:13.639264Z [info     ] log_parsing_start              [src.services.log_parser] path=c:\tmp\Hack2\tests\fixtures\sample_messages.log
2026-02-06T16:29:13.641133Z [info     ] log_parsing_complete           [src.services.log_parser] categories=['other', 'config', 'license', 'os', 'journal'] category_counts={'other': 3, 'config': 4, 'license': 3, 'os': 3, 'journal': 2} path=c:\tmp\Hack2\tests\fixtures\sample_messages.log total_entries=15
2026-02-06T16:29:13.642258Z [info     ] log_patterns_loaded            [src.agents.error_generator] categories=['other', 'config', 'license', 'os', 'journal'] total_entries=15
2026-02-06T16:29:13.685152Z [info     ] agent_execution_start          [agent.error_generator] agent=error_generator input_length=54 trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a
2026-02-06T16:29:13.685152Z [info     ] error_generation_start         [src.agents.error_generator] category=license severity=2 trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a
2026-02-06T16:29

## Demo 2: Multiple Error Categories

Generate errors across all supported categories.

In [5]:
categories = ["config", "license", "os", "journal"]

print("Error Examples by Category:")
print("=" * 80)

for category in categories:
    request = ErrorGenerationRequest(
        error_category=category,
        severity=2
    )
    error = agent.generate_error(request)
    print(f"\n[{category.upper()}]")
    print(error.to_log_format())

Error Examples by Category:
2026-02-06T16:29:16.447092Z [info     ] agent_execution_start          [agent.error_generator] agent=error_generator input_length=53 trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a
2026-02-06T16:29:16.448091Z [info     ] error_generation_start         [src.agents.error_generator] category=config severity=2 trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a
2026-02-06T16:29:16.448091Z [info     ] llm_request_start              [src.utils.llm_client] attempt=1 max_retries=3 model=gpt-4o-2024-08-06 operation=generate_config_error trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-06T16:29:18.314894Z [info     ] llm_token_usage                [src.utils.cost_tracker] cost_usd=0.0021825 daily_usage=1213/500000 input_tokens=521 model=gpt-4o-2024-08-06 operation=generate_config_error output_tokens=88 session_usage=1213/50000 total_tokens=609 trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a wor

## Demo 3: Message Sender Service

Send generated errors to external monitoring systems.

In [6]:
from src.services.message_sender import MessageSenderService

# Initialize sender (endpoint will queue messages if unavailable)
sender = MessageSenderService(output_endpoint="http://localhost:8080/api/messages")

# Send error message
error = agent.generate_error(ErrorGenerationRequest(error_category="license", severity=2))
result = sender.send_message(error)

print("Message Transmission Result:")
print(f"  Status: {result['status']}")
print(f"  Queued: {result.get('queued', False)}")
print(f"\nQueue Size: {sender.get_queue_size()} messages")

2026-02-06T16:29:25.300729Z [info     ] agent_execution_start          [agent.error_generator] agent=error_generator input_length=54 trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a
2026-02-06T16:29:25.301734Z [info     ] error_generation_start         [src.agents.error_generator] category=license severity=2 trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a
2026-02-06T16:29:25.302767Z [info     ] llm_request_start              [src.utils.llm_client] attempt=1 max_retries=3 model=gpt-4o-2024-08-06 operation=generate_license_error trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-06T16:29:26.578787Z [info     ] llm_token_usage                [src.utils.cost_tracker] cost_usd=0.0019725000000000003 daily_usage=3601/500000 input_tokens=529 model=gpt-4o-2024-08-06 operation=generate_license_error output_tokens=65 session_usage=3601/50000 total_tokens=594 trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a workflow_usage=3

## Demo 4: Command Reception

Receive and validate remediation commands from external systems.

In [7]:
from src.services.command_receiver import CommandReceiverService
from src.models.remediation_command import RemediationCommand

# Initialize receiver
receiver = CommandReceiverService()

# Create sample commands
commands = [
    RemediationCommand(
        action_type="config_change",
        target="iris.cpf",
        parameters={"section": "Startup", "key": "globals", "value": "20000"},
        priority="high"
    ),
    RemediationCommand(
        action_type="os_reconfig",
        target="hugepages",
        parameters={"resource_type": "memory", "target_value": 16384},
        priority="medium"
    ),
    RemediationCommand(
        action_type="restart",
        target="instance",
        parameters={"mode": "graceful", "timeout_seconds": 60},
        priority="critical"
    )
]

# Receive commands
print("Receiving Commands:")
print("=" * 80)
for cmd in commands:
    result = receiver.receive_command(cmd.model_dump_json())
    print(f"\n{cmd.action_type.upper()}:")
    print(f"  Status: {result['status']}")
    print(f"  Queue Position: {result['queue_position']}")
    print(f"  Priority: {cmd.priority}")

# Show queue status
status = receiver.get_queue_status()
print(f"\nQueue Status:")
print(f"  Total Commands: {status['total_commands']}")
print(f"  By Priority: {status['by_priority']}")
print(f"  By Action: {status['by_action']}")

Receiving Commands:
2026-02-06T16:29:30.750870Z [info     ] command_receive_start          [src.services.command_receiver] trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a
2026-02-06T16:29:30.751871Z [info     ] command_validated              [src.services.command_receiver] action_type=config_change command_id=45e9e221-a09c-4057-8ec0-e54c5a685daf priority=high trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a
2026-02-06T16:29:30.752872Z [info     ] command_queued                 [src.services.command_receiver] command_id=45e9e221-a09c-4057-8ec0-e54c5a685daf queue_position=1 queue_size=1 trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a

CONFIG_CHANGE:
  Status: accepted
  Queue Position: 1
  Priority: high
2026-02-06T16:29:30.758869Z [info     ] command_receive_start          [src.services.command_receiver] trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a
2026-02-06T16:29:30.759872Z [info     ] command_validated              [src.services.command_receiver] action_type=os_reconfig command_id=bfd36977

## Demo 5: Orchestrator Agent

Route commands to appropriate specialized agents.

In [8]:
from src.agents.orchestrator import OrchestratorAgent

# Initialize orchestrator
orchestrator = OrchestratorAgent()

# Route commands
print("Orchestrator Routing Decisions:")
print("=" * 80)

for cmd in commands:
    response = orchestrator.route_command(cmd)
    print(f"\n{cmd.action_type.upper()} → {response.agent_type.upper()} Agent")
    print(f"  Rationale: {response.rationale}")
    print(f"  Risk: {response.estimated_risk}")
    print(f"  Requires Validation: {response.requires_validation}")

Orchestrator Routing Decisions:
2026-02-06T16:29:30.805758Z [info     ] agent_execution_start          [agent.orchestrator] agent=orchestrator input_length=205 trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a
2026-02-06T16:29:30.805758Z [info     ] orchestration_start            [src.agents.orchestrator] action_type=config_change command_id=45e9e221-a09c-4057-8ec0-e54c5a685daf target=iris.cpf trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a
2026-02-06T16:29:30.806767Z [info     ] llm_request_start              [src.utils.llm_client] attempt=1 max_retries=3 model=gpt-4o-2024-08-06 operation=route_command trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-06T16:29:33.986884Z [info     ] llm_token_usage                [src.utils.cost_tracker] cost_usd=0.002955 daily_usage=4345/500000 input_tokens=598 model=gpt-4o-2024-08-06 operation=route_command output_tokens=146 session_usage=4345/50000 total_tokens=744 trace_

## Demo 6: Config Agent

Execute configuration changes with backup and rollback.

In [9]:
from src.agents.config_agent import ConfigAgent

# Initialize with test CPF file
test_cpf = project_root / "tests" / "fixtures" / "sample_iris.cpf"
config_agent = ConfigAgent(cpf_path=test_cpf)

# Execute config change
response = config_agent.modify_config(
    section="Startup",
    key="globals",
    value="20000"
)

print("Configuration Change Result:")
print("=" * 80)
print(f"  Success: {response.success}")
print(f"  Section: {response.section}")
print(f"  Key: {response.key}")
print(f"  Old Value: {response.old_value}")
print(f"  New Value: {response.new_value}")
print(f"  Requires Restart: {response.requires_restart}")
print(f"  Backup: {response.backup_path}")

2026-02-06T16:29:39.315017Z [info     ] agent_execution_start          [agent.config] agent=config input_length=207 trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a
2026-02-06T16:29:39.316134Z [info     ] config_change_start            [src.agents.config_agent] key=globals section=Startup target_value=20000 trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a
2026-02-06T16:29:39.319716Z [info     ] cpf_read_success               [src.services.cpf_manager] key=globals section=Startup trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a value=10000
2026-02-06T16:29:39.322696Z [info     ] cpf_backup_created             [src.services.cpf_manager] path=c:\tmp\Hack2\tests\fixtures\backups\iris.cpf.backup.20260206_162939 trace_id=f0125a68-e6a4-419d-b3cd-8a190a9c763a
2026-02-06T16:29:39.323716Z [info     ] cpf_write_success              [src.services.cpf_manager] backup_path=c:\tmp\Hack2\tests\fixtures\backups\iris.cpf.backup.20260206_162939 key=globals new_value=20000 old_value=10000 section=Startup trace_id=

## Demo 7: Cost Tracking

View token usage and cost statistics.

In [10]:
from src.utils.cost_tracker import get_cost_tracker

tracker = get_cost_tracker()
stats = tracker.get_stats()

print("Token Usage Statistics:")
print("=" * 80)
print(f"  Total Operations: {stats['total_operations']}")
print(f"  Total Cost: ${stats['total_cost_usd']:.4f}")
print(f"\nWorkflow Budget:")
print(f"  Used: {stats['workflow_used']:,} / {stats['workflow_limit']:,} tokens")
print(f"  Remaining: {stats['workflow_limit'] - stats['workflow_used']:,} tokens")
print(f"\nSession Budget:")
print(f"  Used: {stats['session_used']:,} / {stats['session_limit']:,} tokens")
print(f"\nDaily Budget:")
print(f"  Used: {stats['daily_used']:,} / {stats['daily_limit']:,} tokens")

Token Usage Statistics:
  Total Operations: 9
  Total Cost: $0.0207

Workflow Budget:
  Used: 5,754 / 5,000 tokens
  Remaining: -754 tokens

Session Budget:
  Used: 5,754 / 50,000 tokens

Daily Budget:
  Used: 5,754 / 500,000 tokens


## Summary

This demo has shown:
1. ✅ Error generation for all categories (config, license, os, journal)
2. ✅ Message transmission to external systems with queueing
3. ✅ Command reception and validation with priority ordering
4. ✅ Intelligent orchestration and agent routing
5. ✅ Configuration changes with backup/rollback
6. ✅ Token usage tracking and cost management

### Next Steps
- Explore individual agent notebooks in `notebooks/demos/`
- Review implementation in `src/agents/`
- Run tests with `pytest tests/`
- Check documentation in `specs/001-iris-ops-agent/`